In [1]:
import numpy as np
import anndata

from sklearn.decomposition import PCA
from umap import UMAP
from openTSNE import TSNE

/home/jan/anaconda3/envs/elephant_analysis_env2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load datasets

In [2]:
adata_exut=anndata.read_h5ad("../data/preprocessed/exut_adata.h5ad")
adata_merfish=anndata.read_h5ad("../data/preprocessed/merfish_adata.h5ad")
adata_smartseq=anndata.read_h5ad("../data/preprocessed/smartseq_adata.h5ad")
adata_mnist=anndata.read_h5ad("../data/preprocessed/mnist_adata.h5ad")

datasets = [adata_exut,adata_merfish,adata_smartseq,adata_mnist]

In [3]:
#number of classes per dataset
for ad in datasets:
    print(ad.uns['dataset'],len(np.unique(ad.obs['clusterlabels'])),sep=' ')

exut 19
merfish 25
smartseq 28
mnist 10


In [4]:
def compute_embeddings(ad,seeds):
    
    dataset = ad.uns['dataset']
    
    #compute for scaled and unscaled data
    for use_scaled in [False, True]:
        
        if use_scaled:
            x_hd = ad.layers['X_scaled']
            scaled_str = '_scaled'
        else:
            x_hd = ad.X
            scaled_str = ''
        
        for i,seed in enumerate(seeds):

            print(dataset, 'seed:', seed, 'scaled:', use_scaled)

            pca2 = PCA(random_state=seed,n_components=2)
            pca50 = PCA(random_state=seed,n_components=50)
            umap=UMAP(random_state=seed,verbose=True)
            tsne=TSNE(random_state=seed,verbose=True)

            id_str = f'seed_{seed}'

            print('PCA2')
            x_pca2 = pca2.fit_transform(x_hd)
            print('PCA50')
            x_pca50 = pca50.fit_transform(x_hd)
            print('UMAP')
            x_umap = umap.fit_transform(x_pca50)
            print('TSNE')
            x_tsne = tsne.fit(x_pca50)

            ad.obsm[f'x{scaled_str}_pca2_{id_str}'] = x_pca2
            ad.obsm[f'x{scaled_str}_pca50_{id_str}'] = x_pca50
            ad.obsm[f'x{scaled_str}_umap_{id_str}'] = x_umap
            ad.obsm[f'x{scaled_str}_tsne_{id_str}'] = np.array(x_tsne)

            np.save(f'../results/embeddings/npy/{dataset}_x{scaled_str}_pca2_{id_str}.npy',x_pca2,allow_pickle=False)
            np.save(f'../results/embeddings/npy/{dataset}_x{scaled_str}_pca50_{id_str}.npy',x_pca50,allow_pickle=False)
            np.save(f'../results/embeddings/npy/{dataset}_x{scaled_str}_umap_{id_str}.npy',x_umap,allow_pickle=False)
            np.save(f'../results/embeddings/npy/{dataset}_x{scaled_str}_tsne_{id_str}.npy',x_tsne,allow_pickle=False)
    
    ad.uns['seeds']=seeds
    ad.write_h5ad(f'../results/embeddings/{dataset}_adata_standard_embeddings.h5ad')

In [5]:
###NOTE: elephant embeddings are computed in another conda environment (by Chari & Pachter) and separate notebook
[compute_embeddings(ad=ad,seeds=np.arange(5)) for ad in datasets]

exut seed: 0 scaled: False
PCA2
PCA50
UMAP
UMAP(random_state=0, verbose=True)
Fri Feb 23 21:31:26 2024 Construct fuzzy simplicial set
Fri Feb 23 21:31:26 2024 Finding Nearest Neighbors
Fri Feb 23 21:31:26 2024 Building RP forest with 9 trees


/home/jan/anaconda3/envs/elephant_analysis_env2/lib/python3.11/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


Fri Feb 23 21:31:28 2024 NN descent for 13 iterations
	 1  /  13
	 2  /  13
	 3  /  13
	 4  /  13
	Stopping threshold met -- exiting after 4 iterations
Fri Feb 23 21:31:33 2024 Finished Nearest Neighbor Search
Fri Feb 23 21:31:34 2024 Construct embedding


Epochs completed:   3%| █▌                                                             13/500 [00:00]

	completed  0  /  500 epochs


...

Epochs completed:  92%| ███████████████████████████████████████████████████████▊      183/200 [00:20]

	completed  180  /  200 epochs


Epochs completed: 100%| █████████████████████████████████████████████████████████████ 200/200 [00:22]


Fri Feb 23 22:26:37 2024 Finished embedding
TSNE
--------------------------------------------------------------------------------
TSNE(early_exaggeration=12, random_state=4, verbose=True)
--------------------------------------------------------------------------------
===> Finding 90 nearest neighbors using Annoy approximate search using euclidean distance...
   --> Time elapsed: 26.35 seconds
===> Calculating affinity matrix...
   --> Time elapsed: 1.46 seconds
===> Calculating PCA-based initialization...
   --> Time elapsed: 0.19 seconds
===> Running optimization with exaggeration=12.00, lr=5833.33 for 250 iterations...
Iteration   50, KL divergence 5.9730, 50 iterations in 2.8426 sec
Iteration  100, KL divergence 5.8344, 50 iterations in 2.8418 sec
Iteration  150, KL divergence 5.7859, 50 iterations in 2.8823 sec
Iteration  200, KL divergence 5.7680, 50 iterations in 2.7829 sec
Iteration  250, KL divergence 5.7598, 50 iterations in 2.8432 sec
   --> Time elapsed: 14.19 seconds
===> 

[None, None, None, None]

### Package versions

In [6]:
np.__version__

'1.24.3'

In [7]:
anndata.__version__

'0.10.3'

In [8]:
import sklearn; sklearn.__version__

'1.3.0'

In [9]:
import umap; umap.__version__

'0.5.5'

In [10]:
import openTSNE; openTSNE.__version__

'1.0.1'